In [1]:
import import_ipynb
from demo import *
from pywebio.input import input, FLOAT, input_group
from pywebio.output import put_text, put_processbar, set_processbar, put_file,remove,use_scope,put_collapse,put_image
from pywebio import start_server
import pywebio
import os
from datetime import datetime
import pandas as pd

importing Jupyter notebook from demo.ipynb


In [ ]:
def web_work():
    #put_text(pywebio.session.info)
    
    with use_scope('scope1'):
        put_text('Welcome to Novel cis-element discovery pipeline.\nYou can get familiar with our pipeline and reproduce our experiment with demo setting below：')
        put_text('Reported DNA damage response cis-element')
        put_collapse('Moraxellales DdaA',
        '''
        Input Taxonomy:
        Moraxellales or Acinetobacter

        NCBI Protein ID:
        WP_004925639.1,WP_004928485.1,WP_004920287.1,WP_004923497.1,WP_004922541.1,
        WP_004920289.1,WP_004930057.1,WP_004923206.1,WP_004930164.1,WP_004927788.1,
        WP_004923285.1,WP_004930164.1,WP_004923483.1,WP_004929099.1,WP_011182566.1

        ''')
        
        put_collapse('Deinococcus DdrO',
        '''
        Input Taxonomy:
        Deinococcus

        NCBI Protein ID:
        WP_012695046.1,WP_012694483.1,WP_049760561.1,WP_012692255.1,WP_012693173.1,
        WP_162485426.1,WP_012694690.1,WP_012693159.1,WP_012692693.1,WP_012691930.1,
        WP_012693877.1,WP_012693497.1

        ''')
        
    
        put_text('Unreported DNA damage response cis-element')
        put_collapse('Myxococcota LexA & PafBC',
        '''
        Input Taxonomy:
        Myxococcota 

        NCBI Protein ID:
        WP_011551373.1,WP_011552463.1,WP_011555795.1,WP_011554035.1,
        WP_026113991.1,WP_011551079.1,WP_011551545.1,WP_011554193.1,
        WP_171452263.1,WP_020477747.1,WP_011554445.1,WP_011555315.1

        ''')
        
    put_text('Hope you like it!\nThis website is maintained by shuang_s@zju.edu.cn')
    
    info = input_group("Novel cis-element discovery",[
  input('Input Taxonomy', name='tax'),
  input('Input NCBI Protein ID', name='protein_id')
  #input('Input Representative Species', name='select_specie')
    ])
    
    #put_text(info['tax'],info['protein_id'],info['select_specie'])
    

        
    remove('scope1')
    
    
    #basic information
    ntime=str(datetime.now()).replace(' ','_')
    taxonomy=info['tax'].replace(' ','')
    #select_specie=info['select_specie'].replace(' ','').split('_')[1].split('.')[0]#
    proteins=info['protein_id'].replace(' ','')
    protein_list=proteins.split(',')
    
    check_result=check_before_task(taxonomy,protein_list)
    print(taxonomy)
    #if taxonomy=='Chordata':
    #    check_result='True'
    print(check_result)
    if check_result!='True':
        put_text('Here:'+check_result)
        raise
    put_text('Check passed!')
    put_processbar('bar')
    set_processbar('bar', 0.5 / 10)
    put_text(str(datetime.now()).replace(' ','_'))
    
    #processing
    if taxonomy not in os.listdir('/data/songshuang/server_find_novel_regulator/0_gbk'):
        download_gbk(taxonomy)
        extract_protein(taxonomy)
        diamond_task(taxonomy)
    set_processbar('bar', 2 / 10)
    get_selected_protein_diamond(protein_list,taxonomy,ntime)
    set_processbar('bar', 4 / 10)
    combine_meme_work(ntime)
    set_processbar('bar', 9 / 10)
    motif_for_R(ntime)
    set_processbar('bar', 10 / 10)
    put_text(str(datetime.now()).replace(' ','_'))
    
    #download the result
    os.chdir('/data/songshuang/server_find_novel_regulator/99_User_project/%s/3_meme_result/'%ntime)
    try:
        os.system('tar -czvf %s.tar.gz *'%(taxonomy))
        img = open('/data/songshuang/server_find_novel_regulator/99_User_project/%s/3_meme_result/meme/novel_motif.png'%ntime, 'rb').read()  
        put_image(img)
    except:
        put_text('There is no significant result.')
    try:
        #pal
        put_text('Palindrome sequence')
        img1 = open('/data/songshuang/server_find_novel_regulator/99_User_project/%s/3_meme_result/pal/novel_motif.png'%ntime, 'rb').read()  
        put_image(img1)
    except:
        put_text('There is no significant result.')

    content = open('./%s.tar.gz'%taxonomy, 'rb').read()
    put_file('%s.tar.gz'%taxonomy, content, 'download me')





In [ ]:
if __name__ == '__main__':
    start_server(web_work,port=8848)

Running on all addresses.
Use http://10.72.87.30:8848/ to access the application


RuntimeError: This event loop is already running